In this notebook, I launched 6 fine-tuning jobs, experimenting with: 
- 2 types of instructions, with very different levels of verbosity 
    - simple instruction: """Given the following Airbnb description, Extract the number of bedrooms, determine the type of property, 
            determine whether Is any space shared?, and classify Overall vibes/atmosphere 
            return in JSON format:"""
    - the detailed instruction, not included for space consideration, is highly detailed. It is the same instruction previously provided to
    annotator (in this case, gpt4) to obtain the labels. 
    Explanation: expected benefits of more detailed instructions are that model understands correctly context, nuances and rules but
    length of instructions have implications on compute time  and costs (higher token count)
- 3 models which entail different data preparation, cost and expected performance 
    - babbage-02: smaller and faster model suitable for tasks requiring less understanding of complex contexts. Ideal for cost-effective training and inference.
    - davinci-02: more capable model, excellent for handling nuanced and complex tasks --> might be overkill for our domain (i.e airbnb listings)
    - gpt-3.5-turbo: balances performance and cost, providing a good compromise between the capabilities of Davinci and the efficiency of Babbage.
- hyperparameter turning:
    - n_epoch = 3 (default to auto, which ends up being 3)
    - n_epoch = 4
    Explanation: openai specifies there are 3 hyperparams we can tinker with for the finetuning process, i.e. n_epochs, learning_rate, batch_size.
    I chose to try two different n_epochs for 2 reasons:, 
    1) I noticed the results from earlier fine tuning that the generated text didn't comply with desired structured format
    2) so I looked for openai suggestions, which states "If the model does not follow the training data as much as expected increase the number of epochs by 1 or 2
    This is more common for tasks for which there is a single ideal completion" --> which is our case. Risk of high epochs: overfitting. 
    As for learning rate and batch size, adjusting them having effects on training speed (higher in both cases) but may have poor effects on performance due to skipping 
    over optimal solutions (learning rate) or stable gradient estimates (batch size) --> in our context, it is safe to leave these to default options.

#### fine_tuning_data_v2 (less detailed instructions) n_epochs = 4
- model = babbage-02
- n_epochs = 4
- training_file 'file-4s2gV7Ns8onjSGCOICgoEdEF'
- validation_file 'file-I1PF9wqVH2PHm3pWPyNU7G58'
- fine-tuning_job 'ftjob-ZSkRid0H4hbSl3hYC2LcphOe'
- trained tokens 224,300
- model_name = 'ft:babbage-002:personal::9tf9RTDu'
#### fine_tuning_data_v2_detailedInstructions (detailed instructions) n_epochs = 4
- model = babbage-02
- n_epochs = 4
- training_file 'file-Eiy3m0qnaff0VmGx78Ran3Yh'
- validation_file 'file-sK8gNy3mUGioA7ai7UtAl3MO'
- fine-tuning_job 'ftjob-7ad6DlW1d9g4IEdNkMzzzODa'
- trained_tokens = 785,900
- model_name = 'ft:babbage-002:personal::9tf9VhUQ'

#### fine_tuning_data_v2 (less detailed instructions) n_epochs = 3
- model = babbage-02
- n_epochs = 3
- training_file 'file-4s2gV7Ns8onjSGCOICgoEdEF'
- validation_file 'file-I1PF9wqVH2PHm3pWPyNU7G58'
- fine-tuning_job 'ftjob-tQoI8kM7DzuEqW8xN1tqF8R2'
- trained_tokens 168,225
- model_name = 'ft:babbage-002:personal::9tg6PCpn'

#### fine_tuning_data_v2_detailedInstructions (detailed instructions) n_epochs = 3
- model = babbage-02
- n_epochs = 3
- training_file 'file-Eiy3m0qnaff0VmGx78Ran3Yh'
- validation_file 'file-sK8gNy3mUGioA7ai7UtAl3MO'
- fine-tuning_job 'ftjob-banNk4hagWP3Xy8eQyjjsu79'
- model_name =  'ft:babbage-002:personal::9tg8SIK0'

#### fine_tuning_data_v2_detailedInstructions (detailed instructions) n_epochs = 3
- model = davinci-02
- n_epochs = 3
- training_file 'file-Eiy3m0qnaff0VmGx78Ran3Yh'
- validation_file 'file-sK8gNy3mUGioA7ai7UtAl3MO'
- fine-tuning_job 'ftjob-BGng5tnE2JQpUr2sAQXP8kzf'
- model_name = 'ft:davinci-002:personal::9tgOuHsg'

#### fine_tuning_data_v2_detailedInstructions (detailed instructions) n_epochs = 3
- model = turbo-3.5-turbo
- n_epochs = 3
- training_file 'file-V4lt8WdWRyXaI03BlmSGv6Ar'
- validation_file 'file-N2WqIogEUy228sRTnGlivbMG'
- fine-tuning_job 'ftjob-6DRxt0SWJAA0gjLuZXzFkw4h'
- trained_tokens 603,678
- model_name = 'ft:gpt-3.5-turbo-0125:personal::9tg6PAnc'

In [179]:
import os
import json
import pandas as pd
from collections import defaultdict
from openai import OpenAI

client = OpenAI()

In [56]:
directories = ['fine_tuning_data_v2', 'fine_tuning_data_v2_detailedInstructions']

for directory in directories:
    training_file = client.files.create(
      file=open(f"{directory}/train.jsonl", "rb"),
      purpose="fine-tune"
    )
    
    validation_file = client.files.create(
      file=open(f"{directory}/val.jsonl", "rb"),
      purpose="fine-tune"
    )

    fine_tuning_job = client.fine_tuning.jobs.create(
      training_file=training_file.id,
      validation_file=validation_file.id, 
      model="babbage-002",
        hyperparameters={
        "n_epochs":4
      }
)

    print(f'At directory {directory} \n training_file {training_file} \n validation_file {validation_file} \n and finetuning job {fine_tuning_job}')

At directory fine_tuning_data_v2 
 training_file FileObject(id='file-4s2gV7Ns8onjSGCOICgoEdEF', bytes=277527, created_at=1723052658, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None) 
 validation_file FileObject(id='file-I1PF9wqVH2PHm3pWPyNU7G58', bytes=93709, created_at=1723052659, filename='val.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None) 
 and finetuning job FineTuningJob(id='ftjob-ZSkRid0H4hbSl3hYC2LcphOe', created_at=1723052663, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=4, batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-XDNeT4rDqlxhjhHN3y4zZbkA', result_files=[], seed=1734117511, status='validating_files', trained_tokens=None, training_file='file-4s2gV7Ns8onjSGCOICgoEdEF', validation_file='file-I1PF9wqVH2PHm3pWPyNU7G58', integratio

In [92]:
# we're gonna run this again with no changes to the hyperparams n_epochs

# for fine_tuning_data_v2
fine_tuning_job = client.fine_tuning.jobs.create(
  training_file='file-4s2gV7Ns8onjSGCOICgoEdEF',
  validation_file='file-I1PF9wqVH2PHm3pWPyNU7G58', 
  model="babbage-002",
    )
fine_tuning_job

FineTuningJob(id='ftjob-tQoI8kM7DzuEqW8xN1tqF8R2', created_at=1723056465, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-XDNeT4rDqlxhjhHN3y4zZbkA', result_files=[], seed=595555851, status='validating_files', trained_tokens=None, training_file='file-4s2gV7Ns8onjSGCOICgoEdEF', validation_file='file-I1PF9wqVH2PHm3pWPyNU7G58', integrations=[], user_provided_suffix=None, estimated_finish=None)

In [93]:
#detailed instructions

fine_tuning_job = client.fine_tuning.jobs.create(
  training_file='file-Eiy3m0qnaff0VmGx78Ran3Yh',
  validation_file='file-sK8gNy3mUGioA7ai7UtAl3MO', 
  model="babbage-002",
    )
fine_tuning_job

FineTuningJob(id='ftjob-banNk4hagWP3Xy8eQyjjsu79', created_at=1723056592, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-XDNeT4rDqlxhjhHN3y4zZbkA', result_files=[], seed=1674744380, status='validating_files', trained_tokens=None, training_file='file-Eiy3m0qnaff0VmGx78Ran3Yh', validation_file='file-sK8gNy3mUGioA7ai7UtAl3MO', integrations=[], user_provided_suffix=None, estimated_finish=None)

In [110]:
#detailed instructions
fine_tuning_job = client.fine_tuning.jobs.create(
  training_file='file-Eiy3m0qnaff0VmGx78Ran3Yh',
  validation_file='file-sK8gNy3mUGioA7ai7UtAl3MO', 
  model="davinci-002",
    )
fine_tuning_job

FineTuningJob(id='ftjob-BGng5tnE2JQpUr2sAQXP8kzf', created_at=1723057436, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='davinci-002', object='fine_tuning.job', organization_id='org-XDNeT4rDqlxhjhHN3y4zZbkA', result_files=[], seed=339862684, status='validating_files', trained_tokens=None, training_file='file-Eiy3m0qnaff0VmGx78Ran3Yh', validation_file='file-sK8gNy3mUGioA7ai7UtAl3MO', integrations=[], user_provided_suffix=None, estimated_finish=None)

In [59]:
directories = ['fine_tuning_data_v2_detailedInstructions_gpt3format']

for directory in directories:
    training_file = client.files.create(
      file=open(f"{directory}/train.jsonl", "rb"),
      purpose="fine-tune"
    )
    
    validation_file = client.files.create(
      file=open(f"{directory}/val.jsonl", "rb"),
      purpose="fine-tune"
    )

    fine_tuning_job = client.fine_tuning.jobs.create(
      training_file=training_file.id,
      validation_file=validation_file.id, 
      model="gpt-3.5-turbo"
      
)

    print(f'At directory {directory} \n training_file {training_file} \n validation_file {validation_file} \n and finetuning job {fine_tuning_job}')

At directory fine_tuning_data_v2_detailedInstructions_gpt3format 
 training_file FileObject(id='file-V4lt8WdWRyXaI03BlmSGv6Ar', bytes=1009407, created_at=1723054723, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None) 
 validation_file FileObject(id='file-N2WqIogEUy228sRTnGlivbMG', bytes=337669, created_at=1723054725, filename='val.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None) 
 and finetuning job FineTuningJob(id='ftjob-6DRxt0SWJAA0gjLuZXzFkw4h', created_at=1723054728, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-XDNeT4rDqlxhjhHN3y4zZbkA', result_files=[], seed=201651482, status='validating_files', trained_tokens=None, training_file='file-V4lt8WdWRyXaI03BlmSGv6Ar', validation_fil

In [24]:
client.completions.create(
  model="babbage-002",
  prompt="One of a kind studio apartment in the best Chelsea location. <br /><br />This recently renovated studio apartment is stunning, modern, and well appointed. With a double bed upstairs, a living room and kitchenette downstairs, the bathroom is outside of the apartment in the hallway and shared with one other tenant. <br /><br />Located in a quiet residential building facing the rear, but centrally located near the subway and all the best shopping and restaurants.",
  max_tokens=150,
  temperature=0
)

Completion(id='cmpl-9tbpGITL45DwcP8jG8m1rdmRROKe9', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' <br /><br />The apartment is on the 2nd floor of a 3 story building. <br /><br />The apartment is on the 2nd floor of a 3 story building. <br /><br />The apartment is on the 2nd floor of a 3 story building. <br /><br />The apartment is on the 2nd floor of a 3 story building. <br /><br />The apartment is on the 2nd floor of a 3 story building. <br /><br />The apartment is on the 2nd floor of a 3 story building. <br /><br />The apartment is on the 2nd floor of a 3 story building. <br /><')], created=1723040470, model='babbage-002', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=150, prompt_tokens=91, total_tokens=241))

In [157]:
def call_model(model, prompt, max_tokens=150, temperature=0):

    return client.completions.create(
      # model="ft:babbage-002:personal::9tbdQSFF",
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        frequency_penalty=0,  # Reduce likelihood of repetition
        presence_penalty=0,   # No need to introduce new topics
        stop=["}"],          # Stop generating after two new lines
    )
    
def create_default_data():
    # This returns a defaultdict that defaults to "Not Present" for missing keys
    return defaultdict(lambda: "Not Present")


def fix_and_complete_json(raw_json):
    try:
        # Find the last valid comma and cut the string there
        last_valid_comma = raw_json.rfind(',')
        if last_valid_comma > -1:
            clean_json = raw_json[:last_valid_comma] + "}"
        else:
            clean_json = raw_json

        # Attempt to load it as JSON to see if it is valid
        data = json.loads(clean_json)
        return data
    except json.JSONDecodeError:
        # If still failing, return a message or handle the case as needed
        return "Failed to decode JSON."


def clean_raw_output(raw_output):
    if not raw_output:
        print("Warning: No output received.")
        return None
        
    # Clean up unexpected characters
    clean_output = raw_output.replace('">', '')

    #Find the last valid comma and cut the string there
    last_valid_comma = raw_output.rfind(',')
    if last_valid_comma > -1:
        clean_json = raw_output[:last_valid_comma] + "}"
    else:
        clean_json = raw_output

    return clean_json

def load_clean_json(clean_json):
    # Parse the JSON data
    return json.loads(clean_json)

def parse_output_json(data):
        
    # Prepare a default data container with expected keys defaulted to "Not Present"
    default_data = create_default_data()
    default_data.update(data)  # Update with actual data
    
    # Extract only the expected keys
    expected_keys = ["Number of Bedrooms", "Type of Property", "Is the space shared?", "Overall vibe"]
    validated_data = {key: default_data[key] for key in expected_keys}
    
    return validated_data

In [158]:
# # model = "ft:babbage-002:personal::9tf9VhUQ" #detailed instructions
# # model = 'ft:babbage-002:personal::9tf9RTDu' #less instruction
# # model = 'ft:babbage-002:personal::9tbdQSFF' #different-ish data
# # model = 'ft:gpt-3.5-turbo-0125:personal::9tg6PAnc' #3.5 turbo on detailed instructions
# # model = 'ft:babbage-002:personal::9tg6PCpn'
# model = 'ft:davinci-002:personal::9tgOuHsg'
# # prompt = """
# # Stunning designer Chelsea studio on the best block	One of a kind studio apartment in the best Chelsea location. <br /><br />This recently renovated studio apartment is stunning, modern, and well appointed. With a double bed upstairs, a living room and kitchenette downstairs, the bathroom is outside of the apartment in the hallway and shared with one other tenant. <br /><br />Located in a quiet residential building facing the rear, but centrally located near the subway and all the best shopping and restaurants.
# # """
# # prompt = """
# # extract 4 features from airbnb listing and return response in json format: Bright 2 Bedroom in Astoria	Bright and airy 2 bedroom, steps from Ditmars Blvd N/W train. 5th Floor walkup with roof access and stunning views of NYC.
# # """
# prompt = """
# Welcome to your perfect Brooklyn retreat! Fully furnished studio/junior 1-bedroom apartment in Fort Greene / Clinton Hill only 2 minutes from the C train, 8 minutes from the G train and 10 minutes from Atlantic/Barclays.<br /><br />Apt is in a luxury building with amenities such as a fully equipped kitchen with dishwasher, in-unit washer-dryer, and more. Enjoy a comfortable stay with modern furnishings and all the essentials you need.<br /><br />Please message with a bit info about yourself if interested!
# """

# completion = call_model(model, prompt, max_tokens=50, temperature=0.5)
# raw_output = completion.choices[0].text
# clean_json = clean_raw_output(raw_output)
data = load_clean_json(clean_json)
validated_data = parse_output_json(data)

In [159]:
validated_data

{'Number of Bedrooms': '1',
 'Type of Property': 'Apartment',
 'Is the space shared?': 'Not Present',
 'Overall vibe': 'Not Present'}

In [160]:
clean_json

'{"Number of Bedrooms": "1", "Type of Property": "Apartment", "Is any space shared?": "FALSE", "Overall condition of the property": "MODERN"}'